# Test NIP-66 Relay Monitoring

Interactive notebook for testing the `Nip66` model:
- **RTT** - Round-trip time tests (open, read, write)
- **SSL** - Certificate checks (clearnet only)
- **GEO** - Geolocation lookup (clearnet only)
- **DNS** - DNS resolution (clearnet only)
- **HTTP** - HTTP headers (clearnet or via proxy)
- **Proxy support** - For Tor/I2P/Loki relays (RTT and HTTP tests only)

**GeoIP databases path:** `implementations/bigbrotr/static/`

In [ ]:
import sys
sys.path.insert(0, "../src")

import asyncio
import json
import logging
from pathlib import Path

import geoip2.database
from nostr_sdk import EventBuilder

from models.nip66 import Nip66, Nip66TestError
from models.relay import Relay
from models.keys import Keys
from models.metadata import Metadata

# Enable logging for nip66 module
logging.basicConfig(
    level=logging.DEBUG,
    format="%(asctime)s %(levelname)-5s %(name)s | %(message)s",
    datefmt="%H:%M:%S",
)
logging.getLogger("nip66").setLevel(logging.DEBUG)
logging.getLogger("nostr_sdk").setLevel(logging.WARNING)

print("Imports loaded successfully")

## 1. Setup GeoIP Databases and Keys

In [ ]:
# Load GeoIP databases
geoip_dir = Path("../implementations/bigbrotr/static")

city_db_path = geoip_dir / "GeoLite2-City.mmdb"
asn_db_path = geoip_dir / "GeoLite2-ASN.mmdb"

print(f"City DB exists: {city_db_path.exists()}")
print(f"ASN DB exists: {asn_db_path.exists()}")

city_reader = geoip2.database.Reader(str(city_db_path)) if city_db_path.exists() else None
asn_reader = geoip2.database.Reader(str(asn_db_path)) if asn_db_path.exists() else None

print(f"\nCity reader: {city_reader is not None}")
print(f"ASN reader: {asn_reader is not None}")

In [ ]:
# Generate test keys
keys = Keys.generate()
print(f"Test pubkey: {keys.public_key().to_hex()[:16]}...")

# Create event builder for write tests
event_builder = EventBuilder.text_note("NIP-66 test event")
print(f"Event builder ready")

## 2. Full Test on a Clearnet Relay

In [ ]:
relay = Relay("wss://relay.damus.io")
print(f"Testing: {relay.url}")
print(f"Network: {relay.network}")

In [ ]:
# Run full NIP-66 test (all 5 metadata types)
nip66 = await Nip66.test(
    relay,
    keys=keys,
    event_builder=event_builder,
    city_reader=city_reader,
    asn_reader=asn_reader,
)

print(f"\nMetadata collected:")
print(f"  RTT:  {nip66.rtt_metadata is not None}")
print(f"  SSL:  {nip66.ssl_metadata is not None}")
print(f"  GEO:  {nip66.geo_metadata is not None}")
print(f"  DNS:  {nip66.dns_metadata is not None}")
print(f"  HTTP: {nip66.http_metadata is not None}")

## 3. RTT Metadata

In [ ]:
if nip66.rtt_metadata:
    rtt = nip66.rtt_metadata.data
    print("=== RTT Results ===")
    print(f"rtt_open:  {rtt['rtt_open']} ms")
    print(f"rtt_read:  {rtt['rtt_read']} ms")
    print(f"rtt_write: {rtt['rtt_write']} ms")
else:
    print("No RTT data collected")

## 4. SSL Metadata

In [ ]:
if nip66.ssl_metadata:
    ssl = nip66.ssl_metadata.data
    print("=== SSL Results ===")
    print(f"ssl_valid:      {ssl['ssl_valid']}")
    print(f"ssl_subject_cn: {ssl['ssl_subject_cn']}")
    print(f"ssl_issuer:     {ssl['ssl_issuer']}")
    print(f"ssl_protocol:   {ssl['ssl_protocol']}")
    print(f"ssl_cipher:     {ssl['ssl_cipher']}")
else:
    print("No SSL data (overlay relay or ws://)")

## 5. GEO Metadata

In [ ]:
if nip66.geo_metadata:
    geo = nip66.geo_metadata.data
    print("=== GEO Results ===")
    print(f"geo_ip:         {geo['geo_ip']}")
    print(f"geo_country:    {geo['geo_country']}")
    print(f"geo_city:       {geo['geo_city']}")
    print(f"geo_asn:        {geo['geo_asn']}")
    print(f"geo_asn_org:    {geo['geo_asn_org']}")
    print(f"geo_is_anycast: {geo['geo_is_anycast']}")
else:
    print("No GEO data (overlay relay or no GeoIP DB)")

## 6. DNS Metadata

In [ ]:
if nip66.dns_metadata:
    dns = nip66.dns_metadata.data
    print("=== DNS Results ===")
    print(f"dns_ip:   {dns['dns_ip']}")
    print(f"dns_ipv6: {dns['dns_ipv6']}")
    print(f"dns_ns:   {dns['dns_ns']}")
    print(f"dns_ttl:  {dns['dns_ttl']} seconds")
    print(f"dns_rtt:  {dns['dns_rtt']} ms")
else:
    print("No DNS data (overlay relay)")

## 7. HTTP Metadata

In [ ]:
if nip66.http_metadata:
    http = nip66.http_metadata.data
    print("=== HTTP Results ===")
    print(f"http_server:     {http['http_server']}")
    print(f"http_powered_by: {http['http_powered_by']}")
else:
    print("No HTTP data")

## 8. Full JSON Output

In [ ]:
full_data = {
    "relay_url": nip66.relay.url,
    "generated_at": nip66.generated_at,
    "rtt": nip66.rtt_metadata.data if nip66.rtt_metadata else None,
    "ssl": nip66.ssl_metadata.data if nip66.ssl_metadata else None,
    "geo": nip66.geo_metadata.data if nip66.geo_metadata else None,
    "dns": nip66.dns_metadata.data if nip66.dns_metadata else None,
    "http": nip66.http_metadata.data if nip66.http_metadata else None,
}

print(json.dumps(full_data, indent=2))

## 9. Conversion to RelayMetadata

In [ ]:
# Convert to RelayMetadata for DB storage (5 objects)
rtt_meta, ssl_meta, geo_meta, dns_meta, http_meta = nip66.to_relay_metadata()

print("=== RelayMetadata Objects ===")
for name, meta in [("RTT", rtt_meta), ("SSL", ssl_meta), ("GEO", geo_meta), ("DNS", dns_meta), ("HTTP", http_meta)]:
    if meta:
        print(f"{name}: {meta.metadata_type}")
    else:
        print(f"{name}: None")

## 10. Test Multiple Relays

In [ ]:
relay_urls = [
    "wss://relay.damus.io",
    "wss://nos.lol",
    "wss://relay.nostr.band",
    "wss://nostr.wine",
]

async def test_nip66_safe(url: str) -> tuple[str, Nip66 | None, Exception | None]:
    relay = Relay(url)
    try:
        result = await Nip66.test(
            relay,
            timeout=15.0,
            keys=keys,
            event_builder=EventBuilder.text_note(f"Test {url}"),
            city_reader=city_reader,
            asn_reader=asn_reader,
        )
        return url, result, None
    except Exception as e:
        return url, None, e

results = await asyncio.gather(*[test_nip66_safe(url) for url in relay_urls])

print("=== Results ===")
for url, result, error in results:
    if result:
        rtt = result.rtt_metadata.data if result.rtt_metadata else {}
        geo = result.geo_metadata.data if result.geo_metadata else {}
        print(f"\n{url}")
        print(f"  RTT: {rtt.get('rtt_open', 'N/A')}ms | {geo.get('geo_country', '?')} | {geo.get('geo_asn_org', 'Unknown')}")
    else:
        print(f"\n{url}: ERROR - {type(error).__name__}")

## 11. Test with Proxy (Tor Relay)

**Important**: Only `_test_rtt` and `_test_http` use the proxy. Other tests (SSL, DNS, GEO) are clearnet-only.

To test Tor relays, you need a running Tor proxy (SOCKS5 on port 9050).

In [ ]:
# Example: Test a Tor relay (requires Tor proxy running)
# Uncomment to test if you have Tor running locally

# tor_relay = Relay("ws://oxtrdevav64z64yb7x6rjg3a4a7kblqcmjzreo5hsktyqhmpxsylzead.onion")
# tor_proxy = "socks5://127.0.0.1:9050"
# 
# nip66_tor = await Nip66.test(
#     tor_relay,
#     timeout=30.0,
#     keys=keys,
#     event_builder=event_builder,
#     proxy_url=tor_proxy,  # Required for overlay networks
# )
# 
# # For Tor relay: RTT and HTTP work via proxy, SSL/DNS/GEO return empty
# print(f"RTT: {nip66_tor.rtt_metadata is not None}")  # True (via proxy)
# print(f"HTTP: {nip66_tor.http_metadata is not None}")  # True (via proxy)
# print(f"SSL: {nip66_tor.ssl_metadata is not None}")   # False (clearnet only)
# print(f"DNS: {nip66_tor.dns_metadata is not None}")   # False (clearnet only)
# print(f"GEO: {nip66_tor.geo_metadata is not None}")   # False (clearnet only)

print("Tor test skipped (uncomment to run with local Tor proxy)")

## 12. Proxy Behavior by Test Type

| Test | Uses Proxy | Clearnet | Overlay (with proxy) | Overlay (no proxy) |
|------|------------|----------|---------------------|--------------------|
| RTT  | Yes        | Works    | Works               | Fails              |
| HTTP | Yes        | Works    | Works               | Skipped (empty)    |
| SSL  | No         | Works    | Skipped (empty)     | Skipped (empty)    |
| DNS  | No         | Works    | Skipped (empty)     | Skipped (empty)    |
| GEO  | No         | Works    | Skipped (empty)     | Skipped (empty)    |

## 13. Selective Tests (run_* flags)

In [ ]:
# Test DNS only
nip66_dns_only = await Nip66.test(
    relay,
    run_rtt=False,
    run_ssl=False,
    run_geo=False,
    run_http=False,
)

print("=== DNS Only ===")
print(f"dns_metadata: {nip66_dns_only.dns_metadata is not None}")
if nip66_dns_only.dns_metadata:
    print(f"DNS IP: {nip66_dns_only.dns_metadata.data['dns_ip']}")

In [ ]:
# Test SSL only
nip66_ssl_only = await Nip66.test(
    relay,
    run_rtt=False,
    run_geo=False,
    run_dns=False,
    run_http=False,
)

print("=== SSL Only ===")
print(f"ssl_metadata: {nip66_ssl_only.ssl_metadata is not None}")
if nip66_ssl_only.ssl_metadata:
    print(f"Protocol: {nip66_ssl_only.ssl_metadata.data['ssl_protocol']}")

## 14. Error Handling

In [ ]:
# Test: Missing required parameters for RTT
try:
    result = await Nip66.test(relay, run_rtt=True)  # Missing keys and event_builder!
except ValueError as e:
    print("=== Missing Parameters ===")
    print(f"Correctly raised: {e}")

In [ ]:
# Test: Empty metadata raises error
try:
    nip66_empty = Nip66(
        relay=relay,
        rtt_metadata=Metadata({}),
        ssl_metadata=Metadata({}),
        geo_metadata=Metadata({}),
        dns_metadata=Metadata({}),
        http_metadata=Metadata({}),
    )
except ValueError as e:
    print("=== Empty Metadata ===")
    print(f"Correctly raised: {e}")

## 15. Class Defaults

In [ ]:
print("=== Class Defaults ===")
print(f"_DEFAULT_TEST_TIMEOUT: {Nip66._DEFAULT_TEST_TIMEOUT} seconds")

## 16. Cleanup

## Done!

### Key Points:
- Access fields via `nip66.<type>_metadata.data["key"]`
- All schema keys are present (with `None` for missing)
- Invalid types are silently converted to `None`
- At least one metadata type must have data
- **Only RTT and HTTP tests use `proxy_url`**
- SSL, DNS, GEO are clearnet-only (no proxy support)

### Metadata Types

| Type | Fields | Proxy |
|------|--------|-------|
| RTT  | rtt_open, rtt_read, rtt_write | Yes |
| HTTP | http_server, http_powered_by | Yes |
| SSL  | ssl_valid, ssl_issuer, ssl_protocol, ... | No |
| DNS  | dns_ip, dns_ipv6, dns_ns, dns_ttl, ... | No |
| GEO  | geo_country, geo_city, geo_asn, ... | No |

In [ ]:
relay = Relay("wss://relay.nostr.band")
relay = Relay("wss://nos.lol")
# relay = Relay("wss://relay.damus.io")
# relay = Relay("ws://oxtrdevav64z64yb7x6rjg4ntzqjhedm5b5zjqulugknhzr46ny2qbad.onion")
relay = Relay("wss://sdsdsdsddsdsdfhffrfghjrj.io")
print(f"Testing: {relay.url}")
print(f"Network: {relay.network}")
event = EventBuilder.text_note("NIP-66 test event")
result = await Nip66.test(relay, event_builder=event, keys=keys, timeout=10.0, city_reader=city_reader, asn_reader=asn_reader)

In [ ]:
print(f"\nMetadata collected:")
print(f"  RTT:  {result.rtt_metadata is not None} {result.rtt_metadata.data if result.rtt_metadata else ''}")
print(f"  SSL:  {result.ssl_metadata is not None} {result.ssl_metadata.data if result.ssl_metadata else ''}")
print(f"  GEO:  {result.geo_metadata is not None} {result.geo_metadata.data if result.geo_metadata else ''}")
print(f"  DNS:  {result.dns_metadata is not None} {result.dns_metadata.data if result.dns_metadata else ''}")
print(f"  HTTP: {result.http_metadata is not None} {result.http_metadata.data if result.http_metadata else ''}")